Code from: https://github.com/eriklindernoren/Action-Recognition/

In [1]:
import torch
import sys
import numpy as np
import itertools
from models import *
from dataset import *
from torch.utils.data import DataLoader
from torch.autograd import Variable
import argparse
import time
import datetime

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
#SET PARAMS:
dataset_path = "/var/datasets/videos/UCF-101-frames"
split_path = "/var/datasets/videos/ucfTrainTestlist"
num_epochs = 200
batch_size = 16
sequence_length = 40
img_dim =112
channels = 3
latent_dim = 512
checkpoint_model = "model_checkpoints/ConvLSTM_20.pth"
checkpoint_interval = 10
split_number = 1
num_workers = 4

# Define training set
train_dataset = Dataset(
    dataset_path = dataset_path,
    split_path = split_path,
    split_number = split_number,
    input_shape = (channels, img_dim, img_dim),
    sequence_length = sequence_length,
    training = True,
)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
train_dataset

In [11]:
# Define test set
test_dataset = Dataset(
    dataset_path= dataset_path,
    split_path= split_path,
    split_number=split_number,
    input_shape=(channels, img_dim, img_dim),
    sequence_length= sequence_length,
    training=False,
)

test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [12]:
# Classification criterion
cls_criterion = nn.CrossEntropyLoss().to(device)

# Define network
model = ConvLSTM(
        num_classes=train_dataset.num_classes,
        latent_dim=latent_dim,
        lstm_layers=1,
        hidden_dim=1024,
        bidirectional=True, 
        attention=True,
    )
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [13]:
# Add weights from checkpoint model if specified
if checkpoint_model:
    model.load_state_dict(torch.load(checkpoint_model))

In [14]:
def test_model(epoch):
    """ Evaluate the model on the test set """
    print("")
    model.eval()
    test_metrics = {"loss": [], "acc": []}
    for batch_i, (X, y) in enumerate(test_dataloader):
        image_sequences = Variable(X.to(device), requires_grad=False)
        labels = Variable(y, requires_grad=False).to(device)
        with torch.no_grad():
            # Reset LSTM hidden state
            model.lstm.reset_hidden_state()
            # Get sequence predictions
            predictions = model(image_sequences)
        # Compute metrics
        acc = 100 * (predictions.detach().argmax(1) == labels).cpu().numpy().mean()
        loss = cls_criterion(predictions, labels).item()
        # Keep track of loss and accuracy
        test_metrics["loss"].append(loss)
        test_metrics["acc"].append(acc)
        # Log test performance
        sys.stdout.write(
            "\rTesting -- [Batch %d/%d] [Loss: %f (%f), Acc: %.2f%% (%.2f%%)]"
            % (
                batch_i,
                len(test_dataloader),
                loss,
                np.mean(test_metrics["loss"]),
                acc,
                np.mean(test_metrics["acc"]),
            )
        )
    model.train()
    print("")

In [15]:
# training
for epoch in range(num_epochs):
    epoch_metrics = {"loss": [], "acc": []}
    prev_time = time.time()
    print(f"--- Epoch {epoch} ---")
    for batch_i, (X, y) in enumerate(train_dataloader):
        if X.size(0) == 1:
            continue
        image_sequences = Variable(X.to(device), requires_grad=True)
        labels = Variable(y.to(device), requires_grad=False)
        optimizer.zero_grad()

        # Reset LSTM hidden state
        model.lstm.reset_hidden_state()

        # Get sequence predictions
        predictions = model(image_sequences)

        # Compute metrics
        loss = cls_criterion(predictions, labels)
        acc = 100 * (predictions.detach().argmax(1) == labels).cpu().numpy().mean()
        loss.backward()
        optimizer.step()

        # Keep track of epoch metrics
        epoch_metrics["loss"].append(loss.item())
        epoch_metrics["acc"].append(acc)

        # Determine approximate time left
        batches_done = epoch * len(train_dataloader) + batch_i
        batches_left = num_epochs * len(train_dataloader) - batches_done
        time_left = datetime.timedelta(seconds=batches_left * (time.time() - prev_time))
        prev_time = time.time()

        # Print log
        sys.stdout.write(
            "\r[Epoch %d/%d] [Batch %d/%d] [Loss: %f (%f), Acc: %.2f%% (%.2f%%)] ETA: %s"
            % (
                epoch,
                num_epochs,
                batch_i,
                len(train_dataloader),
                loss.item(),
                np.mean(epoch_metrics["loss"]),
                acc,
                np.mean(epoch_metrics["acc"]),
                time_left,
            )
        )

        # Empty cache
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    # Evaluate the model on the test set
    test_model(epoch)

    # Save model checkpoint
    if epoch % checkpoint_interval == 0:
        os.makedirs("model_checkpoints", exist_ok=True)
        torch.save(model.state_dict(), f"model_checkpoints/{model.__class__.__name__}_{epoch}.pth")

--- Epoch 0 ---
[Epoch 0/200] [Batch 595/597] [Loss: 3.964764 (3.945429), Acc: 68.75% (71.95%)] ETA: 14:27:25.76309833.498706
Testing -- [Batch 236/237] [Loss: 4.438062 (4.200502), Acc: 20.00% (48.63%)]]
--- Epoch 1 ---
[Epoch 1/200] [Batch 595/597] [Loss: 3.960191 (3.935409), Acc: 68.75% (73.13%)] ETA: 14:16:28.28826937.8332430
Testing -- [Batch 236/237] [Loss: 4.418782 (4.199626), Acc: 40.00% (49.30%)]]
--- Epoch 2 ---
[Epoch 2/200] [Batch 595/597] [Loss: 4.034210 (3.923948), Acc: 62.50% (74.02%)] ETA: 14:17:33.13006152.1330903
Testing -- [Batch 236/237] [Loss: 4.268214 (4.191859), Acc: 60.00% (49.86%)]]
--- Epoch 3 ---
[Epoch 3/200] [Batch 595/597] [Loss: 3.852529 (3.914071), Acc: 81.25% (74.80%)] ETA: 14:25:18.841610.56472415
Testing -- [Batch 236/237] [Loss: 4.429952 (4.189671), Acc: 20.00% (49.32%)]]
--- Epoch 4 ---
[Epoch 4/200] [Batch 595/597] [Loss: 4.068864 (3.903917), Acc: 56.25% (75.73%)] ETA: 14:12:10.309851.489681620
Testing -- [Batch 236/237] [Loss: 4.438869 (4.180698), 

[Epoch 40/200] [Batch 595/597] [Loss: 3.767526 (3.674950), Acc: 87.50% (97.28%)] ETA: 15:31:48.4473859354576
Testing -- [Batch 236/237] [Loss: 4.605456 (4.118410), Acc: 0.00% (55.78%)]]]
--- Epoch 41 ---
[Epoch 41/200] [Batch 595/597] [Loss: 3.640451 (3.673838), Acc: 100.00% (97.32%)] ETA: 13:48:33.93239034714027
Testing -- [Batch 236/237] [Loss: 4.523017 (4.114394), Acc: 0.00% (55.91%)]]]
--- Epoch 42 ---
[Epoch 42/200] [Batch 595/597] [Loss: 3.646927 (3.671738), Acc: 100.00% (97.36%)] ETA: 11:41:00.42485889598857
Testing -- [Batch 236/237] [Loss: 4.585792 (4.119794), Acc: 0.00% (55.91%)]]]
--- Epoch 43 ---
[Epoch 43/200] [Batch 595/597] [Loss: 3.701064 (3.670120), Acc: 93.75% (97.48%)] ETA: 19:14:46.340717644047516
Testing -- [Batch 236/237] [Loss: 4.461698 (4.113423), Acc: 20.00% (56.84%)]]
--- Epoch 44 ---
[Epoch 44/200] [Batch 595/597] [Loss: 3.634603 (3.668872), Acc: 100.00% (97.54%)] ETA: 7 days, 13:58:55.9191436
Testing -- [Batch 236/237] [Loss: 4.563685 (4.104018), Acc: 0.00% 

Testing -- [Batch 236/237] [Loss: 4.559785 (4.098782), Acc: 20.00% (56.78%)]]
--- Epoch 80 ---
[Epoch 80/200] [Batch 595/597] [Loss: 3.633687 (3.638550), Acc: 100.00% (99.75%)] ETA: 13:03:40.504793.516312
Testing -- [Batch 236/237] [Loss: 4.581649 (4.111417), Acc: 0.00% (55.20%)]]]
--- Epoch 81 ---
[Epoch 81/200] [Batch 557/597] [Loss: 3.633501 (3.639000), Acc: 100.00% (99.73%)] ETA: 11:30:31.689434729759

KeyboardInterrupt: 